In [8]:
from ioi_utils import *
from sae_variants import *
from training import *
from mandala._next.imports import *
from mandala._next.common_imports import *
from circuit_utils import *

In [3]:
DB_PATH = '/media/amakelov/SanDisk1TB/paper_sprint/test.db'

In [4]:
storage = Storage(db_path=DB_PATH)

In [9]:
storage.preload()
cf = storage.cf(get_logitdiff_loss)
cf.info()

100%|██████████| 3537/3537 [00:06<00:00, 555.34it/s]


ComputationFrame with 8 variable(s) (7191 references to values), 1 operation(s) (3537 references to calls)
Computational graph:


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ output_0 = get_logitdiff_loss(batch_size=batch_size, encoder=encoder, node=node, clean_ld=clean_ld, prompts=pro │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Variables:
    +-----------------------------+------------+
    |             name            | num_values |
    +-----------------------------+------------+
    |          batch_size         |     1      |
    |           encoder           |    3537    |
    |             node            |     38     |
    |           clean_ld          |     1      |
    |           prompts           |     1      |
    | encoder_normalization_scale |     38     |
    |       mean_ablated_ld       |     38     |
    |           output_0          |    3537    |
    +-----------------------------+------------+
Operations:
    +--------------------+-----------+
    |        name        | num_calls |
    +--------------------+-----------+
    | get_logitdiff_loss |    3537   |
    +--------------------+-----------+


In [15]:
cf = storage.cf(train_attribution)

In [17]:
4 * 15 * 55

3300

In [12]:
cf = cf.expand_back(varnames=['encoder'])

100%|██████████| 3537/3537 [00:05<00:00, 592.93it/s]


In [14]:
cf['get_vanilla']

ComputationFrame with 0 variable(s) (0 unique refs), 1 operation(s) (2081 unique calls)
Computational graph:
     = get_vanilla()
Use `.info()` to get more information about the CF, or `.info(*variable_name)`
and `.info(*operation_name)` to get information about specific
variable(s)/operation(s)

In [ ]:
model = get_model()
model.requires_grad_()
MODELS['gpt2small'] = model

In [ ]:
num_floats = 20_000 * 12 * 64 * 19
total_tensor_size_in_mb = num_floats * 4 / 1024 / 1024
print(f'Total tensor size: {total_tensor_size_in_mb:.2f} MB')

In [ ]:
P_train = generate_prompts(
    distribution=full_distribution,
    patterns=['ABB', 'BAB'],
    prompts_per_pattern=10,
    random_seed=0,
)

In [ ]:
activations = {}
grads = {}

def forward_hook(name):
    def hook(model, input, output):
        activations[name] = output
        output.retain_grad()  # Retain gradient for the output
    return hook

def backward_hook(name):
    def hook(grad):
        grads[name] = grad
    return hook


# Register the forward hook
forward_hook_handle = model.blocks[9].attn.hook_z.register_forward_hook(forward_hook('attn_z'))

# Forward pass
input_tensor = prompt_dataset.tokens
output = model(input_tensor)[:, -1, :]

In [ ]:
answer_logits = torch.gather(output, dim=1, index=prompt_dataset.answer_tokens.cuda())
ld = answer_logits[:, 0] - answer_logits[:, 1]

In [ ]:
ld

In [ ]:
# Register the backward hook on the captured activation
activation = activations['attn_z']
backward_hook_handle = activation.register_hook(backward_hook('attn_z'))

individual_gradients = []
for i in tqdm(range(20)): # iterate over the batch
    # Backward pass
    model.zero_grad()
    ld[i].backward(retain_graph=True)
    # The gradient of the loss with respect to the internal activation
    internal_activation_grad = grads['attn_z'][i, :, :, :].detach().clone()
    individual_gradients.append(internal_activation_grad)

# Remove the hooks when done
forward_hook_handle.remove()
backward_hook_handle.remove()


gradients = torch.stack(individual_gradients, dim=0)

In [ ]:
(3 / 20) * 5000

In [ ]:
g = collect_gradients(P_train, 9, 'z')